# IMDB evaluation

In [37]:
import json
import pandas
import os
import io
import random
import json
from nltk.corpus import stopwords
import string
import numpy as np
from scipy import stats

We are going to create the test dataset. We iterate randdomly through the dataset and take equal number of positive reviews than negative reviews

In [38]:
n_files = 1000

file_list_pos = random.sample(os.listdir('aclImdb/test/pos'), n_files)  # select x random files
file_list_neg = random.sample(os.listdir('aclImdb/test/neg'), n_files)

Create the groundtruth lists

In [39]:
list_gt = np.append(np.ones(n_files),np.ones(n_files)*-1)  # ground truth

get the dictionaries

In [40]:
with open('dictionaries/imdb_shuffle_densify.json') as json_file:
    dict_densify = json.load(json_file)
with open('dictionaries/imdb_shuffle_socialsent.json') as json_file:
    dict_socialsent = json.load(json_file)

In [41]:
min(list(dict_densify.values()))

-1.0215262174606323

Function to normalize the dictionary between -1 and 1 values

In [42]:
# def normalize_dict(dictionary):

#     value_list = list(dictionary.values())
#     key_list = list(dictionary.keys())
#     max_list = max(value_list)
#     min_list = min(value_list)
#     a = {key_list[i]: 2 * (value_list[i] - min_list)/(max_list - min_list) -1  for i in range(len(key_list))}
#     return a

In [43]:
# dict_socialsent = normalize_dict(dict_socialsent)
#dict_densify = normalize_dict(dict_densify)

Read each test positive file and calculate the mean value for both densify and socialsent

In [44]:
list_pos_socialsent = np.zeros(n_files)
list_pos_densify = np.zeros(n_files)
stop_words = set(stopwords.words())   # convert into a set for speed purposes
i = 0
for file in file_list_pos:
    with io.open('aclImdb/test/pos/' + file, 'r', encoding='utf8') as review_file:
        review = review_file.read() # read the file
    words = review.split()
    sum_file_socialsent = 0
    sum_file_densify = 0
    counter_socialsent = 0
    counter_densify = 0
    for word in words:
        if word not in stop_words and word not in string.punctuation:
            if dict_socialsent.get(word) is not None:
                sum_file_socialsent += dict_socialsent.get(word)
                counter_socialsent += 1
            if dict_densify.get(word) is not None:    
                sum_file_densify += dict_densify.get(word)
                counter_densify += 1
    list_pos_socialsent[i] = 1 if sum_file_socialsent / counter_socialsent >= 0.5 else -1
    list_pos_densify[i] = 1 if sum_file_densify / counter_densify > 0 else -1
    i += 1
    
    

same for negative files

In [45]:
list_neg_socialsent = np.zeros(n_files)
list_neg_densify = np.zeros(n_files)
stop_words = set(stopwords.words())   # convert into a set for speed purposes
i = 0
for file in file_list_neg:
    with io.open('aclImdb/test/neg/' + file, 'r', encoding='utf8') as review_file:
        review = review_file.read() # read the file
    words = review.split()
    sum_file_socialsent = 0
    sum_file_densify = 0
    counter_socialsent = 0
    counter_densify = 0
    for word in words:
        if word not in stop_words and word not in string.punctuation:
            if dict_socialsent.get(word) is not None:
                sum_file_socialsent += dict_socialsent.get(word)
                counter_socialsent += 1
            if dict_densify.get(word) is not None:    
                sum_file_densify += dict_densify.get(word)
                counter_densify += 1
    list_neg_socialsent[i] = sum_file_socialsent / counter_socialsent
    list_neg_densify[i] = sum_file_densify / counter_densify
    i += 1
    

Join the positive and negative lists

In [46]:
list_socialsent = np.append(list_pos_socialsent, list_neg_socialsent)
list_densify = np.append(list_pos_densify, list_neg_densify)

Evaluate with pearson coefficient

In [47]:
evaluation_socialsent = stats.pearsonr(list_gt, list_socialsent)
evaluation_densify = stats.pearsonr(list_gt, list_densify)

In [48]:
print('Socialsent: ', evaluation_socialsent)
print('Densify: ', evaluation_densify)

Socialsent:  (0.9938703851511883, 0.0)
Densify:  (0.9892299984151911, 0.0)


In [49]:
np.min(list_socialsent)

0.40526596495164907